# Homework w05d04 - Feature selection


### Read in the Wisconsin Breast Cancer Dataset

### Assign the columns

The attributes below will be the columns of the dataset.
  
Attribute                     

1. Sample code number [subject ID]
1. Class
1. Cell nucleus mean radius
1. Cell nucleus SE radius
1. Cell nucleus worst radius
1. Texture mean
1. Texture SE
1. Texture worst
1. Perimeter mean
1. Perimeter SE
1. Perimeter worst
1. Area mean
1. Area SE
1. Area worst
1. Smoothness mean
1. Smoothness SE
1. Smoothness worst
1. Compactness mean
1. Compactness SE
1. Compactness worst
1. Concavity mean
1. Concavity SE
1. Concavity worst
1. Concave points mean
1. Concave points SE
1. Concave points worst
1. Symmetry mean
1. Symmetry SE
1. Symmetry worst
1. Fractal dimension mean
1. Fractal dimension SE
1. Fractal dimension worst

The column names are taken from the dataset info file.
For more information check out the information file: wdbc.names.

In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.metrics import accuracy_score, average_precision_score, f1_score, recall_score, precision_score
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data", header=None)

column_names = ['id','malignant',
                'nucleus_mean','nucleus_se','nucleus_worst',
                'texture_mean','texture_se','texture_worst',
                'perimeter_mean','perimeter_se','perimeter_worst',
                'area_mean','area_se','area_worst',
                'smoothness_mean','smoothness_se','smoothness_worst',
                'compactness_mean','compactness_se','compactness_worst',
                'concavity_mean','concavity_se','concavity_worst',
                'concave_pts_mean','concave_pts_se','concave_pts_worst',
                'symmetry_mean','symmetry_se','symmetry_worst',
                'fractal_dim_mean','fractal_dim_se','fractal_dim_worst']

df.columns = column_names

# Define the feature matrix X
X = df.iloc[:,2:]

#Define the response y and recode it numerically
y = df.malignant
y = y.map(lambda x: 0 if x == "B" else 1)
print y.head()
X.head()

0    1
1    1
2    1
3    1
4    1
Name: malignant, dtype: int64


,nucleus_mean,nucleus_se,nucleus_worst,texture_mean,texture_se,texture_worst,perimeter_mean,perimeter_se,perimeter_worst,area_mean,...,concavity_worst,concave_pts_mean,concave_pts_se,concave_pts_worst,symmetry_mean,symmetry_se,symmetry_worst,fractal_dim_mean,fractal_dim_se,fractal_dim_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


#### 1. Use the standard scaler to transform the feature matrix X, put the output into a pandas data frame and assign the appropriate column names

In [2]:
from sklearn.preprocessing import StandardScaler
Xt = StandardScaler().fit_transform(X)
Xt = pd.DataFrame(Xt,columns = column_names[2:])
Xt.head()

,nucleus_mean,nucleus_se,nucleus_worst,texture_mean,texture_se,texture_worst,perimeter_mean,perimeter_se,perimeter_worst,area_mean,...,concavity_worst,concave_pts_mean,concave_pts_se,concave_pts_worst,symmetry_mean,symmetry_se,symmetry_worst,fractal_dim_mean,fractal_dim_se,fractal_dim_worst
0,1.097064,-2.073335,1.269934,0.984375,1.568466,3.283515,2.652874,2.532475,2.217515,2.255747,...,1.886690,-1.359293,2.303601,2.001237,1.307686,2.616665,2.109526,2.296076,2.750622,1.937015
1,1.829821,-0.353632,1.685955,1.908708,-0.826962,-0.487072,-0.023846,0.548144,0.001392,-0.868652,...,1.805927,-0.369203,1.535126,1.890489,-0.375612,-0.430444,-0.146749,1.087084,-0.243890,0.281190
2,1.579888,0.456187,1.566503,1.558884,0.942210,1.052926,1.363478,2.037231,0.939685,-0.398008,...,1.511870,-0.023974,1.347475,1.456285,0.527407,1.082932,0.854974,1.955000,1.152255,0.201391
3,-0.768909,0.253732,-0.592687,-0.764464,3.283553,3.402909,1.915897,1.451707,2.867383,4.910919,...,-0.281464,0.133984,-0.249939,-0.550021,3.394275,3.893397,1.989588,2.175786,6.046041,4.935010
4,1.750297,-1.151816,1.776573,1.826229,0.280372,0.539340,1.371011,1.428493,-0.009560,-0.562450,...,1.298575,-1.466770,1.338539,1.220724,0.220556,-0.313395,0.613179,0.729259,-0.868353,-0.397100


#### 2. Use best subset selection to determine the 5 best predictors

In [3]:
from sklearn.feature_selection import SelectKBest, f_classif
selector = SelectKBest(f_classif, k=5)
selector.fit(Xt,y)
mask = selector.get_support(indices=True)
print(Xt[mask].columns.tolist())

['nucleus_worst', 'perimeter_se', 'concavity_worst', 'concave_pts_se', 'fractal_dim_mean']


#### 3. Use percentile based feature selection, vary the percentile by 5% steps and indicate which features are included.

In [4]:
from sklearn.feature_selection import SelectPercentile#, VarianceThreshold, f_classif

# Use e.g.
for p in range(5,101,5):
    selector = SelectPercentile(f_classif, percentile=p)
    selector.fit(Xt,y)
    mask = selector.get_support(indices=True)
    print("Percentile %d, selected features: %d" % (p, len(mask)))
    #print(Xt[mask].columns.tolist())


Percentile 5, selected features: 2
Percentile 10, selected features: 3
Percentile 15, selected features: 5
Percentile 20, selected features: 6
Percentile 25, selected features: 8
Percentile 30, selected features: 9
Percentile 35, selected features: 11
Percentile 40, selected features: 12
Percentile 45, selected features: 14
Percentile 50, selected features: 15
Percentile 55, selected features: 16
Percentile 60, selected features: 18
Percentile 65, selected features: 19
Percentile 70, selected features: 21
Percentile 75, selected features: 22
Percentile 80, selected features: 24
Percentile 85, selected features: 25
Percentile 90, selected features: 27
Percentile 95, selected features: 28
Percentile 100, selected features: 30


#### 4. Use feature ranking with recursive feature elimination and cross-validated selection for feature selection. Vary the number of cross validations between 2 and 10. How does the number of features change?

In [5]:
from sklearn.feature_selection import RFECV, RFE
from sklearn.linear_model import LogisticRegression

# use e.g. RFECV(estimator, step=1,cv=5)
for cv in range(2,11,1):
    selector = RFECV(LogisticRegression(),step=1,cv=cv)
    selector.fit(Xt,y)
    mask = selector.get_support(indices=True)
    print("# Folds %d, selected features: %d" % (cv, len(mask)))
    #print(Xt[mask].columns.tolist())

# Folds 2, selected features: 23
# Folds 3, selected features: 23
# Folds 4, selected features: 30
# Folds 5, selected features: 30
# Folds 6, selected features: 25
# Folds 7, selected features: 17
# Folds 8, selected features: 20
# Folds 9, selected features: 27
# Folds 10, selected features: 30


The usualy recommanded number of cross validation folds, 5 and 10, correspond to the highest number of features selected.

#### 5. Feed RFECV with cv=5 into grid search CV with logistic regression. Vary the regularisation parameters C and the penalty (l1 or l2) and obtain the scores for the best model. Obtain the regression coefficients and determine the five with largest absolute value.

In [6]:
from sklearn.model_selection import GridSearchCV

estimator = LogisticRegression()
selector = RFECV(estimator, step=1, cv=5)

params = {'estimator__penalty':['l1','l2'],
'estimator__C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
}

model = GridSearchCV(selector, param_grid = params)
model_fit = model.fit(X, y)

In [8]:
model_fit.best_params_

{'estimator__C': 10, 'estimator__penalty': 'l1'}